[View in Colaboratory](https://colab.research.google.com/github/PanthonImem/KNN-Recommedation/blob/master/lab_mf_for_completed.ipynb)

In [0]:
# required library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [19]:

df = pd.read_csv('https://raw.githubusercontent.com/PanthonImem/Content-Based-Lab-Notebook/master/userLog_201801_201802_for_participants.csv', delimiter = ';', error_bad_lines = False, low_memory = False)
df.head(3)

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018,1,1,0


In [20]:
test_users = pd.read_csv('https://raw.githubusercontent.com/PanthonImem/KNN-Recommedation/master/200_users.csv', delimiter = ';', error_bad_lines = False, low_memory = False)
test_users.head(3)

,userCode
0,003b2f18-60b4-37be-c63d-0caead1afa97
1,02b44422-25f6-56bb-459c-5944b346badb
2,0419a38a-d883-3bb7-fa1a-8e1a256b89d3


In [21]:
df_train = df[df['userCode'].isin(set(test_users['userCode']))]
df_train['interacted'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
# Convert the dataframe into a matrix using pivot() and set index of df_train for fast search 
df_train_indexed = df_train[['userCode', 'project_id', 'interacted']].drop_duplicates()
df_train_pivot = (df_train_indexed.pivot(index = 'userCode', columns = 'project_id', values = 'interacted')
                                  .fillna(0))
df_train_pivot.head(3)

project_id,192,271,291,368,934,1093,1903,1908,1936,1967,...,9352,9367,9378,9385,9389,9426,9428,9447,9477,9483
userCode,,,,,,,,,,,,,,,,,,,,,
003b2f18-60b4-37be-c63d-0caead1afa97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02b44422-25f6-56bb-459c-5944b346badb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0419a38a-d883-3bb7-fa1a-8e1a256b89d3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_train_matrix = df_train_pivot.values
df_train_matrix.shape

(200, 477)

In [24]:
df_train_indexed = df_train_indexed.set_index('userCode')
df_train_indexed.head(3)

,project_id,interacted
userCode,,
7efda6e1-5990-f0f7-9cee-2ea77afcc4b2,6529,1
04da975f-e725-5008-70f2-0506905a1a96,4386,1
4f91b4c7-f4af-4247-4867-14ad6fa99335,9018,1


## Implementing matrix factorisation for recommender system
For more infomation, http://nimfa.biolab.si/

In [25]:
!pip3 install nimfa
import nimfa

In [26]:
snmf = nimfa.Snmf(df_train_matrix, seed="random_vcol", rank=70, max_iter=12,rcond = None, rel_error=1e-5)
snmf_fit = snmf()

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:605: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  K = np.linalg.lstsq(CtC, CtA)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

...

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

In [0]:
W = snmf_fit.basis()
H = snmf_fit.coef()

In [28]:
result = (W*H)
result.shape

(200, 477)

In [0]:
colnames = df_train_pivot.columns
user_list = list(df_train_pivot.index)

In [30]:
result_df = pd.DataFrame(result)
result_df['userCode'] = user_list
result_df = result_df.set_index(['userCode'])
result_df.columns = colnames
result_df.head(3)

project_id,192,271,291,368,934,1093,1903,1908,1936,1967,...,9352,9367,9378,9385,9389,9426,9428,9447,9477,9483
userCode,,,,,,,,,,,,,,,,,,,,,
003b2f18-60b4-37be-c63d-0caead1afa97,3.195838e-18,1.140836e-15,3.965983e-16,1.146036e-15,5.065836e-16,6.491155e-16,5.749134e-20,2.306380e-16,3.181574e-18,8.504425e-16,...,1.948871e-19,4.461544e-16,2.306380e-16,1.435301e-21,8.504425e-16,1.124704e-15,5.547197e-16,2.953117e-16,3.965983e-16,6.932447e-22
02b44422-25f6-56bb-459c-5944b346badb,3.083764e-17,1.168478e-15,4.242401e-16,1.173678e-15,5.342254e-16,6.767573e-16,2.769929e-17,2.582798e-16,3.082337e-17,8.780843e-16,...,2.783669e-17,4.737962e-16,2.582798e-16,2.764324e-17,8.780843e-16,1.152346e-15,5.823615e-16,3.229535e-16,4.242401e-16,2.764249e-17
0419a38a-d883-3bb7-fa1a-8e1a256b89d3,8.664932e-17,1.224290e-15,4.800518e-16,1.229490e-15,5.900371e-16,7.325690e-16,8.351097e-17,3.140915e-16,8.663506e-17,9.338959e-16,...,8.364837e-17,5.296079e-16,3.140915e-16,8.345492e-17,9.338959e-16,1.208157e-15,6.381732e-16,3.787651e-16,4.800518e-16,8.345418e-17


#### Recommended items

In [0]:
def get_item_interacted(df, user_id):
        # to ignore interacted itemset when recommend
    """
    Args:
    - userId = user id
    Return:
    - set of interacted itemsets
    """
    interacted_projects = df[df.index == user_id][['project_id']]
    return set(interacted_projects['project_id'].values)

In [32]:
recommend_df = pd.DataFrame(columns = ['userCode', 'project_id'])
recommend_df

,userCode,project_id


In [0]:
for i in user_list:
    
    user_id = i
    items_to_ignore = get_item_interacted(df_train_indexed, user_id)
    pred_df = result_df[result_df.index == user_id].reset_index()
    pred_df = pd.melt(pred_df, id_vars=["userCode"], var_name="project_id", value_name="rating")
    pred_df = (pred_df[~pred_df['project_id'].isin(items_to_ignore)]
                                             .sort_values(['rating'], ascending=False)
                                             .head(7))
    pred_df = pred_df[['userCode', 'project_id']]
    recommend_df = pd.concat([recommend_df, pred_df])

In [34]:
recommend_df.head(10)

,userCode,project_id
387,003b2f18-60b4-37be-c63d-0caead1afa97,8715
104,003b2f18-60b4-37be-c63d-0caead1afa97,4507
302,003b2f18-60b4-37be-c63d-0caead1afa97,7920
322,003b2f18-60b4-37be-c63d-0caead1afa97,8112
210,003b2f18-60b4-37be-c63d-0caead1afa97,6161
421,003b2f18-60b4-37be-c63d-0caead1afa97,9083
22,003b2f18-60b4-37be-c63d-0caead1afa97,2402
18,02b44422-25f6-56bb-459c-5944b346badb,2291
176,02b44422-25f6-56bb-459c-5944b346badb,5560
236,02b44422-25f6-56bb-459c-5944b346badb,6447
